In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import Dataset, DatasetDict
import pandas as pd

# Load DeBERTa model and tokenizer
model_name = "microsoft/deberta-v3-base"  
model = TFAutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

df = pd.read_csv("/kaggle/input/augclaimrag1/AugClaimWithRAG1.csv")
df.dropna()
df = df.dropna(subset=['claim'])  
df = df.dropna(subset=['label'])  
df = df.dropna(subset=['abstract'])  

from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset
dataset_train = Dataset.from_pandas(train_df)
dataset_val = Dataset.from_pandas(val_df)

# Create DatasetDict
dataset = DatasetDict({
    'train': dataset_train,
    'validation': dataset_val
})

def tokenize(batch):
    # Tokenize claims and abstracts with padding and truncation
    claim_tokens = tokenizer(batch["claim"], padding=True, truncation=True, max_length=256, return_tensors="tf")
    abstract_tokens = tokenizer(batch["abstract"], padding=True, truncation=True, max_length=256, return_tensors="tf")

    return {
        "claim_input_ids": claim_tokens["input_ids"].numpy(),  # Convert to numpy array
        "claim_attention_mask": claim_tokens["attention_mask"].numpy(),  # Convert to numpy array
        "abstract_input_ids": abstract_tokens["input_ids"].numpy(),  # Convert to numpy array
        "abstract_attention_mask": abstract_tokens["attention_mask"].numpy(),  # Convert to numpy array
        "label": batch["label"]  # Keep labels as they are
    }

dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

# Set the dataset format
dataset_encoded.set_format('tf', columns=[
    'claim_input_ids', 'claim_attention_mask', 'abstract_input_ids', 'abstract_attention_mask', 'label'
])

# Setting BATCH_SIZE
BATCH_SIZE = 4

def order(inp):
    data = list(inp.values())
    return {
        'claim_input_ids': data[1],
        'claim_attention_mask': data[2],
        'abstract_input_ids': data[3],
        'abstract_attention_mask': data[4]
    }, data[0]

train_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['train'][:])
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000).map(order, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['validation'][:])
val_dataset = val_dataset.batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)


class DeBERTaForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Process claim inputs
        claim_output = self.bert(
            input_ids=inputs['claim_input_ids'],
            attention_mask=inputs['claim_attention_mask']
        )
        # Extract the hidden state or pooled output (adjust based on the model output)
        claim_pooled_output = claim_output.last_hidden_state[:, 0, :]  # First token (CLS)

        # Process abstract inputs
        abstract_output = self.bert(
            input_ids=inputs['abstract_input_ids'],
            attention_mask=inputs['abstract_attention_mask']
        )
        abstract_pooled_output = abstract_output.last_hidden_state[:, 0, :]  # First token (CLS)

        # Combine outputs and pass through the classification layer
        combined_output = tf.concat([claim_pooled_output, abstract_pooled_output], axis=-1)
        return self.fc(combined_output)


classifier = DeBERTaForClassification(model, num_classes=3)
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Training the model
history = classifier.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100  # Use fewer epochs for testing; increase for final training
)




In [ ]:
# Plotting functions remain the same
import matplotlib.pyplot as plt
import os

# Plotting accuracy over epochs
def plot_accuracy(history, save_dir="/kaggle/working", filename="accDEBERTA.png"):
    os.makedirs(save_dir, exist_ok=True)
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Validation Accuracy Over Epochs')
    plt.legend(loc="lower right")
    plt.grid(True)
    save_path = os.path.join(save_dir, filename)
    plt.savefig(filename)
    plt.show()
    print(f"Accuracy plot saved to {save_path}")

plot_accuracy(history)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Extract true labels from the test set
true_labels = np.array(dataset_encoded['validation']['label'])

# Predict labels for the test dataset
predictions = classifier.predict(val_dataset)
predicted_labels = np.argmax(predictions, axis=1)  # Get the class with the highest probability

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Display the confusion matrix as a heatmap
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot(cmap=plt.cm.Blues)
plt.figure(figsize=(8, 8))
disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
plt.title("Confusion Matrix")
plt.savefig("cm.png")
plt.show()